# 🛡️ VisionGuard

**VisionGuard** é uma aplicação web para **monitoramento inteligente por vídeo**, com detecção em tempo real de **objetos cortantes** e identificação da presença de **pessoas na cena**, permitindo classificar contextualmente os alarmes como:

- 🔪 **Objeto cortante detectado**
- 🧍‍♂️🔪 **Pessoa com objeto cortante detectado**

A aplicação pode ser usada com a **webcam ao vivo** ou por **vídeos enviados** via upload. Alarmes são exibidos na interface e enviados por e-mail com imagem, além da geração de **relatórios automáticos em PDF**.

Neste notebook, vamos realizar o treinamento de um modelo YOLO (You Only Look Once) para detecção de objetos cortantes. Utilizando um dataset específico, o objetivo é treinar uma rede neural para identificar e localizar facas e outros objetos cortantes em imagens.

A primeira parte do código instala o YOLOv8, preparando o ambiente para tarefas de detecção de objetos cortantes.

A segunda parte monta o Google Drive, permitindo o acesso e armazenamento de arquivos necessários para o projeto.

Ao final do treinamento, o modelo gerado será salvo no formato .pt (PyTorch), pronto para ser utilizado na aplicação VisionGuard, para a detecção de objetos cortantes em imagens ao vivo ou vídeos.

#Etapa 1: Instalar o YOLOv8 (Ultralytics)

In [ ]:
!pip install ultralytics --quiet
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.7 MB/s eta 0:00:00
Mounted at /content/drive


#👇 Etapa 2: Upload do dataset ZIP manualmente (use a barra lateral do Colab ou este código)

In [ ]:
from google.colab import files  #Importa o módulo files do pacote google.colab, que fornece funcionalidades para interagir com arquivos no ambiente do Google Colab.

print("👉 Envie o arquivo ZIP do seu dataset (ex: knife-dataset-new.zip)")
uploaded = files.upload() #o Método files.upload() permite que você faça upload de arquivos do seu computador local para o ambiente do Colab.


👉 Envie o arquivo ZIP do seu dataset (ex: knife-dataset-new.zip)


Saving knife-dataset-new.zip to knife-dataset-new.zip


#✅ Etapa 3: Descompactar o dataset

In [ ]:
import zipfile  #zipfile: módulo da biblioteca padrão do Python que permite trabalhar com arquivos ZIP.
import os    #os: módulo que fornece funções para interagir com o sistema operacional, como listar diretórios.

# Pega o nome do arquivo enviado, uploaded é um dicionário contendo os arquivos enviados pelo usuário. O next(iter(uploaded)) obtém o nome do primeiro arquivo enviado.
dataset_zip = next(iter(uploaded))


# Extrai o conteúdo :Abre o arquivo ZIP em modo de leitura ('r').Extrai todo o conteúdo do arquivo ZIP para o diretório /content/.O uso do with garante que o arquivo ZIP será fechado corretamente após a extração.
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall("/content/")

# Verifica o que foi extraído
print("🗂️ Conteúdo da pasta /content/:")
print(os.listdir("/content/"))

# Define o nome correto da pasta extraída
dataset_path = "/content/"

# Verifica subpastas internas
print("📁 Subpastas no dataset:")
print(os.listdir(dataset_path))


NameError: name 'uploaded' is not defined

# ✅ Etapa 4: Treinar modelo YOLOv8 com seu dataset


In [ ]:
from ultralytics import YOLO #Importa a classe YOLO da biblioteca Ultralytics, que fornece funcionalidades para treinamento, validação e inferência de modelos YOLOv8.

# Carrega o modelo pré-treinado YOLOv8 Small (yolov8s.pt). Este modelo serve como ponto de partida para o treinamento com seu dataset personalizado.
#Você pode optar por modelos de diferentes tamanhos, como:
#yolov8n.pt (Nano): Mais rápido, porém menos preciso.
#yolov8m.pt (Medium): Compromisso entre velocidade e precisão.
#yolov8l.pt (Large) e yolov8x.pt (Extra Large): Mais precisos, porém mais lentos e exigentes em recursos computacionais.
model = YOLO("yolov8s.pt")


# Inicia o treinamento
model.train(
    data=f"{dataset_path}/data.yaml",   #data: Caminho para o arquivo data.yaml, que deve conter as configurações do seu dataset, como caminhos para as imagens de treinamento e validação, além das classes.
    epochs=50,#epochs: Define o número de vezes que o modelo irá percorrer todo o dataset durante o treinamento. Mais épocas podem levar a um modelo mais preciso, mas também aumentam o tempo de treinamento.
    imgsz=640,#imgsz: Especifica o tamanho das imagens de entrada. O padrão é 640x640 pixels, mas pode ser ajustado conforme necessário.
    batch=16,#batch: Determina quantas imagens serão processadas simultaneamente durante o treinamento. Um valor maior pode acelerar o treinamento, mas requer mais memória GPU.
    project="knife-detection",#project: Nome do projeto, utilizado para organizar os resultados e checkpoints do treinamento.
    name="yolov8-train"#name: Nome específico para esta execução do treinamento, permitindo diferenciar entre diferentes experimentos dentro do mesmo projeto.
)


ModuleNotFoundError: No module named 'ultralytics'

# ✅ Etapa 5: Avaliar desempenho no conjunto de validação

In [ ]:
metrics = model.val()  #Este comando utiliza o método val() da classe YOLO para avaliar o desempenho do modelo no conjunto de validação.
                       #O método processa as imagens de validação e calcula diversas métricas que ajudam a entender a eficácia do modelo treinado.
print(metrics)


Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1010.1±362.0 MB/s, size: 22.9 KB)


val: Scanning /content/valid/labels.cache... 797 images, 0 backgrounds, 0 corrupt: 100%|██████████| 797/797 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 50/50 [00:11<00:00,  4.32it/s]


                   all        797        849      0.945      0.843      0.927       0.58
Speed: 0.7ms preprocess, 9.0ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to knife-detection/yolov8-train2
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x792134546210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.03203

📊 Principais Métricas de Avaliação do YOLOv8

Precisão(Precision): Proporção de detecções corretas entre todas as detecções feitas pelo modelo.​

Revocação (Recall): Capacidade do modelo de identificar todas as instâncias relevantes nas imagens.​

mAP@0.5 (Mean Average Precision @ IoU 0.5): Média da precisão considerando uma sobreposição (IoU) de 50% entre as caixas preditas e as reais.​

mAP@0.5:0.95: Média da precisão calculada em múltiplos limiares de IoU, variando de 0.50 a 0.95, em incrementos de 0.05. Fornece uma visão abrangente do desempenho do modelo em diferentes níveis de rigor.​

Métricas de Velocidade: Tempo gasto em diferentes fases do processo de validação, como pré-processamento, inferência e pós-processamento.
